[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fw-ai/cookbook/blob/main/examples/function_calling/fw_autogen_image_gen.ipynb)

## Preable - Install Deps

There are only a few dependencies for this tutorial.

In [ ]:
!pip install pyautogen openai fireworks-ai

## Introduction - Art Generator

In this example we will use AutoGen framework to construct an agent that is capable of generating an image through DALLE-3 and saving it to local disk.

For this demo, we are going to utilize [function calling](https://readme.fireworks.ai/docs/function-calling) feature launched by Fireworks. We initialize two agents - `UserProxyAgent` and `AssistantAgent`. The `AssistantAgent` is given the ability to issue a call for the provided functions but not execute them while `UserProxyAgent` is given the ability to execute the function calls issues by the `AssistantAgent`. In order to achieve this behaviour we use decorators provided by AutoGen library called `register_for_llm` and `register_for_execution`. Using these decorators allows us to easily define python functions and turn them into [JSON Spec](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat) needed by function calling API.

Finally, we setup system prompt for both the agents. We ask the `AssistantAgent` to be a helpful agent & focus on generating the correct function calls and we leave `UserProxyAgent` as is. For more advanced use cases we can ask `UserProxyAgent` to be a plan generator.


In [13]:
import autogen
import asyncio
import os
from IPython import get_ipython
from typing_extensions import Annotated
from typing import List
import uuid
import requests  # to perform HTTP requests
from pathlib import Path
from openai import OpenAI
from matplotlib import pyplot as plt
import cv2
import fireworks.client
from fireworks.client.image import ImageInference, Answer

## Setup

In order to use the Fireworks AI function calling model, you must first obtain Fireworks API Keys. If you don't already have one, you can one by following the instructions [here](https://readme.fireworks.ai/docs/quickstart). Replace FW_API_KEY with your obtained key.

In [4]:
FW_API_KEY = "YOUR_API_KEY"

config_list = [
  {
    "model": "accounts/fireworks/models/firefunction-v1",
    "api_key": FW_API_KEY,
    "base_url": "https://api.fireworks.ai/inference/v1",
    "temperature": 0.0
  }
]

## Configure Tools

For this notebook, we are going to use 2 sets of tools
1. **Image Generation** - We will use [StableDiffusion XL](https://fireworks.ai/models/fireworks/stable-diffusion-xl-1024-v1-0) model on Fireworks platform to generate images for us given the prompt. The tool itself would save the file to a randomly generated file name.
2. **Show Image** - This tool, given a valid file path, will display the image.


Using the AutoGen framework we demonstrate the co-operative nature of agents working with each other to accomplish a complex task. This tutorial can be extended to perform more complicated tasks such as generating stock price charts etc.

In [28]:
llm_config = {
    "config_list": config_list,
    "timeout": 120,
    "temperature": 0
}
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="You are a helpful assistant that can use available functions when needed to solve problems. At each point, do your best to determine if the user's request has been addressed. If the request HAS been addressed, respond with a summary of the result. The summary must be written as a coherent helpful response to the user request e.g. 'Sure, here is result to your request ' or 'The tallest mountain in Africa is ..' etc. The summary MUST end with the word TERMINATE. If the  user request is pleasantry or greeting, you should respond with a pleasantry or greeting and TERMINATE.",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE."),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"},
)


@user_proxy.register_for_execution()
@chatbot.register_for_llm(name="generate_and_save_images", description="Function to paint, draw or illustrate images based on the users query or request. Generates images from a given query using OpenAI's DALL-E model and saves them to disk. Use the code below anytime there is a request to create an image.")
async def generate_and_save_images(query: Annotated[str, "A natural language description of the image to be generated"], image_size: Annotated[str, "The size of the image to be generated. (default is '1024x1024')"] = "1024x1024") -> List[str]:
    """
    :param query: A natural language description of the image to be generated.
    :param image_size: )
    :return: A list of filenames for the saved images.
    """
    fireworks.client.api_key = FW_API_KEY
    inference_client = ImageInference(model="stable-diffusion-xl-1024-v1-0")

    # List to store the file names of saved images
    saved_files = []


    file_name = str(uuid.uuid4()) + ".jpg"  # Assuming the image is a JPG
    file_path = Path(file_name)

    # Generate an image using the text_to_image method
    answer : Answer = await inference_client.text_to_image_async(
        prompt=query,
        cfg_scale=7,
        height=1024,
        width=1024,
        sampler=None,
        steps=30,
        seed=0,
        safety_check=False,
        output_image_format="JPG",
        # Add additional parameters here as necessary
    )

    if answer.image is None:
      raise RuntimeError(f"No return image, {answer.finish_reason}")
    else:
      answer.image.save(file_path)
      print(f"Image saved to {file_path}")
      saved_files.append(str(file_path))

    # Return the list of saved files
    return {"path": saved_files[0]}

# Example usage of the function:
# generate_and_save_images("A cute baby sea otter")

@user_proxy.register_for_execution()
@chatbot.register_for_llm(name="show_image", description="A function that is capable for displaying an image given path to a image file in png or jpg or jpeg.")
def show_image(path: Annotated[str, "The path to the image file that needs to be displayed"]) -> str:
  img = cv2.imread(path,-1)
  plt.imshow(img)
  plt.axis("off")
  plt.show()
  return ""


## Initiating Chat

Now we will use the `initiate_chat` functionality to give our AutoGen bot a complex task to accomplish. For this particular task - we ask it to paint an image of ethiopian coffee and show it's image.

In [29]:
# start the conversation
await user_proxy.a_initiate_chat(
    chatbot,
    message="paint and show an image of a glass of ethiopian coffee, freshly brewed in a tall glass cup, on a table right in front of a lush green forest scenery.",
)

user_proxy (to chatbot):

paint and show an image of a glass of ethiopian coffee, freshly brewed in a tall glass cup, on a table right in front of a lush green forest scenery.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

 
***** Suggested tool Call (call_F18NYriOOks268hACJ1inp9V): generate_and_save_images *****
Arguments: 
{"query": "a glass of ethiopian coffee, freshly brewed in a tall glass cup, on a table right in front of a lush green forest scenery", "image_size": "1024x1024"}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING ASYNC FUNCTION generate_and_save_images...
Image saved to aace2257-dd6f-45bb-83dd-75c4fe0cf4c7.jpg
user_proxy (to chatbot):

user_proxy (to chatbot):

***** Response from calling tool "call_F18NYriOOks268hACJ1inp9V" *****
{"path": "aace2257-dd6f-45bb-83dd-75

ChatResult(chat_history=[{'content': 'paint and show an image of a glass of ethiopian coffee, freshly brewed in a tall glass cup, on a table right in front of a lush green forest scenery.', 'role': 'assistant'}, {'content': ' ', 'tool_calls': [{'id': 'call_F18NYriOOks268hACJ1inp9V', 'function': {'arguments': '{"query": "a glass of ethiopian coffee, freshly brewed in a tall glass cup, on a table right in front of a lush green forest scenery", "image_size": "1024x1024"}', 'name': 'generate_and_save_images'}, 'type': 'function', 'index': 0}], 'role': 'assistant'}, {'content': '{"path": "aace2257-dd6f-45bb-83dd-75c4fe0cf4c7.jpg"}', 'tool_responses': [{'tool_call_id': 'call_F18NYriOOks268hACJ1inp9V', 'role': 'tool', 'content': '{"path": "aace2257-dd6f-45bb-83dd-75c4fe0cf4c7.jpg"}'}], 'role': 'tool'}, {'content': ' Sure, here is the result of your request. I have generated an image of a glass of Ethiopian coffee, freshly brewed in a tall glass cup, on a table right in front of a lush green f